<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# FEC - Création du dataset "Charges"

**Tags:** #fec #finance #snippet #operation #charges

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** Ce notebook permet de créer le dataset pour visualiser les charges entre l'année N et N-1.

## Input

### Import libraries

In [ ]:
import naas_data_product

### Setup Variables

In [ ]:
# Inputs
input_folder_path = "/home/ftp/FEC-engine/outputs/FEC/bdd_fin_enr"

# Outputs
output_folder_path = "/home/ftp/FEC-engine/outputs/FEC/dataset_charges"
prompt_data = """
Voici le jeu de données du détail des charges d'une entreprise entre une ou plusieurs années avec:
- le type de charge dans la colonne 'LABEL'
- les valeurs de l'année N en colonne 'VALUE'
- les valeurs de l'année N en colonne 'VALUE_N-1'
Peux-tu me donner un bref aperçu factuel de l'évolution des charges et identifier les variations les plus importantes entre l'année N et N-1?
Peux-tu commencer ton analyse par "Les charges de l'entreprise de l'entreprise ont ..."?
"""

## Model

### Récupération du dernier fichier input

In [ ]:
df_input = get_last_df(input_folder_path)
print("✅ Row fetched:", len(df_input))
df_input.head(1)

### Création du dataset "CHARGES"

In [ ]:
def prep_data(
    df_init,
    label):
    # Init
    df = df_init.copy()

    # Filtre RUBRIQUE_N0 = CHARGES
    df = df[df["RUBRIQUE_N0"] == "CHARGES"]
    
    # Groupby + agg
    to_group = ["ENTITY", "PERIOD", label]
    to_agg = {"VALUE": "sum", "VALUE_N-1": "sum"}
    to_rename = {label: "LABEL"}
    df = df.groupby(to_group, as_index=False).agg(to_agg).rename(columns=to_rename)

    # Mettre en valeur positive VALUE
    df["VALUE"] = df["VALUE"].abs()
    df["VALUE_N-1"] = df["VALUE_N-1"].abs()
    df["VARV"] = df["VALUE"] - df["VALUE_N-1"]
    df["VARP"] = df["VARV"] / df["VALUE_N-1"]
    df["VALUE_D"] = (df["VALUE"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")
    df["VALUE_N-1_D"] = (df["VALUE_N-1"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")
    df["VARV_D"] = (df["VARV"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")
    df.loc[df["VARV"] > 0, "VARV_D"] = "+" + df["VARV_D"]
    df["VARP_D"] = (df["VARP"]).map("{:,.1%}".format)
    df.loc[df["VARP"] > 0, "VARP_D"] = "+" + df["VARP_D"]
    return df

data_prep = prep_data(df_input, label="RUBRIQUE_N1")
print("Row fetched:", len(data_prep))
data_prep.head(12)

### Création des analyses par entité

In [ ]:
def create_data_charges(df_init):
    # Init
    df = df_init.copy()
    df_output = pd.DataFrame()
    
    # Get periods by entity
    entities = df[["ENTITY"]].drop_duplicates(ignore_index=True)
    for row in entities.itertuples():
        entity = row.ENTITY
        tmp_df = df.copy()
        tmp_df = tmp_df[(tmp_df["ENTITY"] == entity)]
        
        # Get analysis
        analysis = get_ia_analysis(tmp_df, prompt_data)
        tmp_df["AI_ANALYSIS"] = analysis
            
        # Concat
        df_output = pd.concat([df_output, tmp_df])
    return df_output.reset_index(drop=True)

df_charges = create_data_charges(data_prep)
print("Row fetched:", len(df_charges))
df_charges.head(24)

## Output

### Sauvegarde des fichiers en csv

In [ ]:
save_df(df_charges, output_folder_path)